In [ ]:
import os
import pandas as pd 
import glob

cwd = os.getcwd()
results_dir = f"{cwd}/results"

In [ ]:
letters = ["a","b","c","d","e"]
model_names = ["microsoft/deberta-v3-base","FacebookAI/xlm-roberta-base","dslim/bert-base-NER-uncased"]# ["answerdotai/ModernBERT-base"]
best_params_dct = {l:{mn:[] for mn in model_names} for l in letters}
for l in letters:
    for mn in model_names:
        df = pd.read_csv(f"{cwd}/results/{l}/sghead/hyptuning_{mn.split('/')[-1]}.csv")
        el_vals = df["values_0"]
        f1_vals = df["values_1"]
        id_besteval = list(el_vals).index(el_vals.min())
        id_bestf1 = list(f1_vals).index(f1_vals.max())
        if id_besteval == id_bestf1:
            best_params_dct[l][mn].append({
                "eval_loss": df.loc[id_besteval, "values_0"],
                "overall_f1": df.loc[id_besteval, "values_1"],
                "batch_size": df.loc[id_besteval, "params_batch_size"],
                "lr": df.loc[id_besteval, "params_lr"],
                "max_length": df.loc[id_besteval, "params_max_length"],
                "weight_decay": df.loc[id_besteval, "params_weight_decay"]
            })
        else:
            for idx in [id_besteval, id_bestf1]:
                best_params_dct[l][mn].append({
                    "eval_loss": df.loc[idx, "values_0"],
                    "overall_f1": df.loc[idx, "values_1"],
                    "batch_size": df.loc[idx, "params_batch_size"],
                    "lr": df.loc[idx, "params_lr"],
                    "max_length": df.loc[idx, "params_max_length"],
                    "weight_decay": df.loc[idx, "params_weight_decay"]
                })
best_params_dct

In [12]:
el_vals = df["values_0"]
f1_vals = df["values_1"]

print(el_vals.min(), f1_vals.max())
print(list(el_vals).index(el_vals.min()), list(f1_vals).index(f1_vals.max()))
df.iloc[5]

0.3422350794076919 0.6062322946175638
5 5


Unnamed: 0                                                        5
number                                                            5
values_0                                                   0.342235
values_1                                                   0.606232
datetime_start                           2026-02-19 23:59:19.612194
datetime_complete                        2026-02-20 00:02:22.288162
duration                                     0 days 00:03:02.675968
params_batch_size                                                 8
params_lr                                                    0.0002
params_max_length                                               256
params_weight_decay                                         0.07455
system_attrs_NSGAIISampler:generation                             0
state                                                      COMPLETE
Name: 5, dtype: object

In [15]:
df.iloc[[5, 6, 9, 11, 16, 21, 24, 26]]

,Unnamed: 0,number,values_0,values_1,datetime_start,datetime_complete,duration,params_batch_size,params_lr,params_max_length,params_weight_decay,system_attrs_NSGAIISampler:generation,state
5,5,5,0.342235,0.606232,2026-02-19 23:59:19.612194,2026-02-20 00:02:22.288162,0 days 00:03:02.675968,8,0.000200,256,0.074550,0,COMPLETE
6,6,6,0.349024,0.565610,2026-02-20 00:02:22.289119,2026-02-20 00:07:42.885841,0 days 00:05:20.596722,8,0.000066,512,0.013451,0,COMPLETE
9,9,9,0.366925,0.525376,2026-02-20 00:19:19.510936,2026-02-20 00:25:23.063015,0 days 00:06:03.552079,32,0.000185,512,0.089817,0,COMPLETE
11,11,11,0.380471,0.522280,2026-02-20 00:31:15.419041,2026-02-20 00:36:12.225143,0 days 00:04:56.806102,8,0.000050,256,0.079142,0,COMPLETE
16,16,16,0.368861,0.544962,2026-02-20 00:59:05.603885,2026-02-20 01:02:08.525661,0 days 00:03:02.921776,8,0.000106,512,0.098567,0,COMPLETE
21,21,21,0.369268,0.534701,2026-02-20 01:25:26.669672,2026-02-20 01:31:30.828811,0 days 00:06:04.159139,32,0.000275,512,0.075243,0,COMPLETE
24,24,24,0.358257,0.577707,2026-02-20 01:43:41.796942,2026-02-20 01:47:30.457506,0 days 00:03:48.660564,8,0.000101,512,0.097176,0,COMPLETE
26,26,26,0.362504,0.569178,2026-02-20 01:53:34.044074,2026-02-20 01:58:41.943548,0 days 00:05:07.899474,8,0.000095,512,0.072571,0,COMPLETE
